<a href="https://colab.research.google.com/github/AlexVonSchwerdtner/BA780-Team6/blob/main/Boston_Crime_Scene_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Reading all files to the environment

In [3]:
crimes2015 = pd.read_csv('https://raw.githubusercontent.com/AlexVonSchwerdtner/BA780-Team6/main/crime-incident-reports-2015.csv')
crimes2016 = pd.read_csv('https://raw.githubusercontent.com/AlexVonSchwerdtner/BA780-Team6/main/crime-incident-reports-2016.csv')
crimes2017 = pd.read_csv('https://raw.githubusercontent.com/AlexVonSchwerdtner/BA780-Team6/main/crime-incident-reports-2017.csv')
crimes2018 = pd.read_csv('https://raw.githubusercontent.com/AlexVonSchwerdtner/BA780-Team6/main/crime-incident-reports-2018.csv')
crimes2019 = pd.read_csv('https://raw.githubusercontent.com/AlexVonSchwerdtner/BA780-Team6/main/crime-incident-reports-2019.csv')
crimes2020 = pd.read_csv('https://raw.githubusercontent.com/AlexVonSchwerdtner/BA780-Team6/main/crime-incident-reports-2020.csv')
crimes2021 = pd.read_csv('https://raw.githubusercontent.com/AlexVonSchwerdtner/BA780-Team6/main/crime-incident-reports-2021.csv')
crimes2015.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I192068249,2647,Other,THREATS TO DO BODILY HARM,B2,280,NaN,2015-08-28 10:20:00,2015,8,Friday,10,Part Two,WASHINGTON ST,42.330119,-71.084251,"(42.33011862, -71.08425106)"
1,I192061894,1106,Confidence Games,FRAUD - CREDIT CARD / ATM FRAUD,C11,356,NaN,2015-08-20 00:00:00,2015,8,Thursday,0,Part Two,CHARLES ST,42.300605,-71.061268,"(42.30060543, -71.06126785)"
2,I192038828,1107,Fraud,FRAUD - IMPERSONATION,A1,172,NaN,2015-11-02 12:24:00,2015,11,Monday,12,Part Two,ALBANY ST,42.334288,-71.072395,"(42.33428841, -71.07239518)"
3,I192008877,1107,Fraud,FRAUD - IMPERSONATION,E18,525,NaN,2015-07-31 10:00:00,2015,7,Friday,10,Part Two,WINGATE RD,42.237009,-71.129566,"(42.23700950, -71.12956606)"
4,I182090828,1102,Fraud,FRAUD - FALSE PRETENSE / SCHEME,D4,159,NaN,2015-12-01 12:00:00,2015,12,Tuesday,12,Part Two,UPTON ST,42.342432,-71.072258,"(42.34243222, -71.07225766)"


In [16]:
crime_all=pd.concat([crimes2015,crimes2016,crimes2017,crimes2018,crimes2019,crimes2020,crimes2021]).drop_duplicates().reset_index(drop=True)
print(crime_all.shape)
print(crimes2015.shape[0]+crimes2016.shape[0]+crimes2017.shape[0]+crimes2018.shape[0]+crimes2019.shape[0]+crimes2020.shape[0]+crimes2021.shape[0])

(493638, 17)
494281
